In [49]:
#improting libraries 
import numpy as np 
import torch 
import random 
import math

In [50]:
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.label = label 
        self._prev = set(_children)
        self._op = _op
        self.grad = 0.0
        self._backward = lambda: None
    
    def __repr__(self):
        return f"Value({self.data})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
            
        out._backward = _backward
        return out
        
    def __radd__(self, other):
        return self + other
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        out._backward = _backward
        return out
    
    def __rmul__(self, other):
        return self * other
    
    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        return self + -other
    
    def __rsub__(self, other):
        return self - other
    def tanh(self):
        x = self.data 
        t = ((math.exp(2*x) - 1) / (math.exp(2*x) + 1))
        out = Value(t, (self, ), 'tanh')
        
        def _backward():
            self.grad += (1 - t** 2) * out.grad
        
        out._backward = _backward        
        return out 
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp') 
        
        def _backward():
            self.grad += out.data * out.grad
            
        out._backward = _backward
        return out 
    
    def __pow__(self, other):
        assert isinstance(other, (int, float))
        out = Value(self.data ** other, (self, ), f'**{other}')
        
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        return out 
    
    def __truediv__(self, other):
        return self * other ** -1 
        
    def backward(self):
        self.grad = 1.0 
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        for node in reversed(topo):
            node._backward()
    

In [51]:
class Neuron():
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
    
    def __call__(self, x):
        act = sum((xi * wi for xi, wi in zip(x, self.w)), self.b)
        out = act.tanh()
        return out 
    
    def parameters(self):
        return self.w + [self.b]
        

In [52]:
class Layer():
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neroun in self.neurons for p in neroun.parameters()]

In [53]:
class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts 
        self.layers = [Layer(sz[i], sz[i + 1]) for i in range(len(nouts))]
        
    def __call__(self, x):
        for layers in self.layers:
            x = layers(x)
        return x 
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [54]:
#defining an example
n = MLP(3, [3, 3, 1])

xs = [
[2.0, 3.0, -1.0],
[3.0, -1.0, 0.5],
[0.5, 1.0, 1.0],
[1.0, 1.0, -1.0]]
ys = [1.0, -.80, -.9650, 1.0] # desired targets
ypred = [n(x) for x in xs]
ypred

[Value(-0.48150073165942253),
 Value(-0.4616237771610382),
 Value(-0.49653185933963107),
 Value(-0.3404285852319617)]

In [55]:
#gradient descend 
for _ in range(100000):
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt) ** 2 for yout, ygt in zip(ys, ypred))
    
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()
    
    for p in n.parameters(): 
        p.data += -0.05 * p.grad
    
    print(loss)

Value(4.325554077010908)
Value(3.3764275678182973)
Value(3.2134943673260055)
Value(3.0696802922360464)
Value(2.9061748851525984)
Value(2.712410686047109)
Value(2.456848611930794)
Value(2.012669228997998)
Value(1.2109845098507641)
Value(0.6414191115410873)
Value(0.40192598186582745)
Value(0.28124631286956236)
Value(0.2111707280232647)
Value(0.16646013342554955)
Value(0.13595221558177117)
Value(0.11406457193974676)
Value(0.09774001465763825)
Value(0.0851827595442783)
Value(0.07527754046095055)
Value(0.06729962510737758)
Value(0.06076005098012778)
Value(0.05531834256548576)
Value(0.05073095143933119)
Value(0.04681957292332342)
Value(0.043451003644980515)
Value(0.04052395169117002)
Value(0.037960172272006515)
Value(0.03569837189621065)
Value(0.03368992968527304)
Value(0.03189583851245402)
Value(0.030284481716634844)
Value(0.02882999273105166)
Value(0.02751102816548308)
Value(0.026309838613687894)
Value(0.025211556840114377)
Value(0.024203646715113346)
Value(0.023275472424154273)
Value(0.02

KeyboardInterrupt: 

In [56]:
ypred, ys

([Value(0.9986174327083884),
  Value(-0.799996865000604),
  Value(-0.9650027855160955),
  Value(0.9985554932087027)],
 [1.0, -0.8, -0.965, 1.0])